# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end

aluminium_setup (generic function with 2 methods)

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trival statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298647758788                   -0.85    4.0
  2   -8.300230146240       -2.80       -1.25    1.6
  3   -8.300450113332       -3.66       -1.87    3.4
  4   -8.300462165346       -4.92       -2.68    2.4
  5   -8.300464270574       -5.68       -3.19    3.4
  6   -8.300464451577       -6.74       -3.31   15.9
  7   -8.300464526594       -7.12       -3.42    1.0
  8   -8.300464583108       -7.25       -3.55    1.0
  9   -8.300464634911       -7.29       -3.82    1.1
 10   -8.300464641264       -8.20       -3.98    1.8
 11   -8.300464643617       -8.63       -4.35    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.64238816801                   -0.70    5.1
  2   -16.64613125617       -2.43       -1.14    1.1
  3   -16.67921822224       -1.48       -1.87    3.4
  4   -16.67928105000       -4.20       -2.69    5.0
  5   -16.67928605650       -5.30       -3.00    4.2
  6   -16.67928621209       -6.81       -3.32    1.9
  7   -16.67928622180       -8.01       -3.79    1.0
  8   -16.67928622406       -8.65       -4.49    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged, n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32597813341                   -0.56    6.4
  2   -33.33285995714       -2.16       -1.00    1.0
  3   -33.33548158864       -2.58       -1.69    2.9
  4   -33.33681722618       -2.87       -2.25    4.2
  5   -33.33692627756       -3.96       -2.69    9.9
  6   -33.33694374868       -4.76       -3.63    2.2
  7   -33.33694386433       -6.94       -3.90   11.2
  8   -33.33694392637       -7.21       -4.82    2.2


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298554865558                   -0.85    4.0
  2   -8.300286764292       -2.76       -1.59    1.5
  3   -8.300457471264       -3.77       -2.63    6.1
  4   -8.300413979980   +   -4.36       -2.41   12.1
  5   -8.300464523815       -4.30       -3.71    2.9
  6   -8.300464597151       -7.13       -3.98    5.1
  7   -8.300464643844       -7.33       -5.09    3.1


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing(), n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32560952256                   -0.56    6.5
  2   -33.32582094531       -3.67       -1.27    1.2
  3   -22.84353996853   +    1.02       -0.56    8.0
  4   -33.27581296484        1.02       -1.64    9.8
  5   -33.31299966542       -1.43       -1.54    4.0
  6   -33.32407097624       -1.96       -1.86    2.8
  7   -32.93631116490   +   -0.41       -1.26    4.5
  8   -33.33459973642       -0.40       -2.25    4.8
  9   -33.33557384750       -3.01       -2.42    2.4
 10   -33.33655826555       -3.01       -2.48    2.6
 11   -33.33685719516       -3.52       -3.00    2.0
 12   -33.33693353238       -4.12       -3.36    3.8
 13   -33.33694008869       -5.18       -3.54    3.0
 14   -33.33694255422       -5.61       -3.71    2.9
 15   -33.33694324020       -6.16       -4.10    3.4


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.